In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler

from sklearn_pandas import DataFrameMapper

from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import Dense, Dropout

import keras

from keras.callbacks import EarlyStopping

In [2]:
pd.set_option("max.columns", None)

In [3]:
df = pd.read_csv("mixed_scores_teams.csv")

In [4]:
df.head()

,Unnamed: 0,date,team,rank_assists,rank_blocks,rank_deffeciency,rank_drebounds,rank_fouls,rank_ft_pct,rank_oeffeciency,rank_opp_assists,rank_opp_blocks,rank_opp_drebounds,rank_opp_fouls,rank_opp_orebounds,rank_opp_ptsfastbreak,rank_opp_ptsfrom2,rank_opp_ptsfrom3,rank_opp_ptsinpaint,rank_opp_ptspergame,rank_opp_steals,rank_opp_threepointpct,rank_opp_turnovers,rank_opp_twopointpct,rank_orebounds,rank_ptsfrom2,rank_ptsfrom3,rank_ptsfromfastbreak,rank_ptsinpaint,rank_scoring,rank_steals,rank_threeptpct,rank_turnovers,rank_twoptpct,start (et),visitor,pts_visitor,home,pts_home,ot,home_game,back_to_back
0,0,2015-11-03,Atlanta Hawks,7,23,15,16,5,11,15,25,23,18,30,29,18,3,29,2,10,13,21,9,2,26,13,16,4,15,21,5,21,11,5,7:30p,Atlanta Hawks,98,Miami Heat,92,NaN,False,False
1,1,2015-11-04,Atlanta Hawks,5,26,12,16,3,12,16,15,24,20,27,29,17,3,26,6,8,17,16,4,7,22,12,13,6,10,22,5,20,11,8,8:00p,Brooklyn Nets,87,Atlanta Hawks,101,NaN,True,True
2,2,2015-11-06,Atlanta Hawks,3,20,10,16,3,13,15,16,25,18,26,29,15,3,24,6,6,19,11,4,10,26,12,14,6,12,17,3,18,11,8,8:00p,Atlanta Hawks,121,New Orleans Pelicans,115,NaN,False,False
3,3,2015-11-07,Atlanta Hawks,3,18,15,16,3,9,7,22,27,14,24,27,17,7,24,6,10,19,10,6,16,26,9,14,6,8,13,2,12,14,5,7:30p,Washington Wizards,99,Atlanta Hawks,114,NaN,True,True
4,4,2015-11-09,Atlanta Hawks,2,20,11,16,2,4,6,21,24,14,24,25,17,9,25,11,10,23,11,3,18,27,10,7,6,8,6,2,9,13,7,8:00p,Minnesota Timberwolves,117,Atlanta Hawks,107,NaN,True,False


In [5]:
df.tail()

,Unnamed: 0,date,team,rank_assists,rank_blocks,rank_deffeciency,rank_drebounds,rank_fouls,rank_ft_pct,rank_oeffeciency,rank_opp_assists,rank_opp_blocks,rank_opp_drebounds,rank_opp_fouls,rank_opp_orebounds,rank_opp_ptsfastbreak,rank_opp_ptsfrom2,rank_opp_ptsfrom3,rank_opp_ptsinpaint,rank_opp_ptspergame,rank_opp_steals,rank_opp_threepointpct,rank_opp_turnovers,rank_opp_twopointpct,rank_orebounds,rank_ptsfrom2,rank_ptsfrom3,rank_ptsfromfastbreak,rank_ptsinpaint,rank_scoring,rank_steals,rank_threeptpct,rank_turnovers,rank_twoptpct,start (et),visitor,pts_visitor,home,pts_home,ot,home_game,back_to_back
12217,403,2021-02-25,Washington Wizards,13,30,27,14,30,23,22,10,11,26,1,20,4,25,24,7,29,17,25,11,18,17,7,24,26,14,9,15,29,16,17,9:00p,Washington Wizards,112,Denver Nuggets,110,NaN,False,False
12218,404,2021-02-27,Washington Wizards,14,30,27,15,30,22,22,10,12,26,1,23,4,21,25,6,29,17,25,12,17,17,6,24,26,14,9,15,29,16,18,7:00p,Minnesota Timberwolves,112,Washington Wizards,128,NaN,True,False
12219,405,2021-02-28,Washington Wizards,14,30,27,14,30,22,22,10,14,26,1,25,4,21,24,6,29,18,25,12,16,17,6,24,27,13,7,15,28,17,18,7:00p,Washington Wizards,110,Boston Celtics,111,NaN,False,True
12220,406,2021-03-02,Washington Wizards,16,30,25,15,30,21,22,10,13,25,1,25,4,23,22,8,29,19,26,9,16,18,7,24,26,13,7,14,29,17,18,7:00p,Memphis Grizzlies,125,Washington Wizards,111,NaN,True,False
12221,407,2021-03-04,Washington Wizards,15,30,26,14,30,21,22,10,13,24,1,25,4,25,22,8,29,21,26,11,17,18,7,24,26,13,9,14,28,19,16,7:00p,Los Angeles Clippers,117,Washington Wizards,119,NaN,True,False


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12222 entries, 0 to 12221
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              12222 non-null  int64 
 1   date                    12222 non-null  object
 2   team                    12222 non-null  object
 3   rank_assists            12222 non-null  int64 
 4   rank_blocks             12222 non-null  int64 
 5   rank_deffeciency        12222 non-null  int64 
 6   rank_drebounds          12222 non-null  int64 
 7   rank_fouls              12222 non-null  int64 
 8   rank_ft_pct             12222 non-null  int64 
 9   rank_oeffeciency        12222 non-null  int64 
 10  rank_opp_assists        12222 non-null  int64 
 11  rank_opp_blocks         12222 non-null  int64 
 12  rank_opp_drebounds      12222 non-null  int64 
 13  rank_opp_fouls          12222 non-null  int64 
 14  rank_opp_orebounds      12222 non-null  int64 
 15  ra

In [7]:
del df["Unnamed: 0"], df["date"], df["team"], df["start (et)"], df["visitor"], df["home"], df["ot"]

In [8]:
df

,rank_assists,rank_blocks,rank_deffeciency,rank_drebounds,rank_fouls,rank_ft_pct,rank_oeffeciency,rank_opp_assists,rank_opp_blocks,rank_opp_drebounds,rank_opp_fouls,rank_opp_orebounds,rank_opp_ptsfastbreak,rank_opp_ptsfrom2,rank_opp_ptsfrom3,rank_opp_ptsinpaint,rank_opp_ptspergame,rank_opp_steals,rank_opp_threepointpct,rank_opp_turnovers,rank_opp_twopointpct,rank_orebounds,rank_ptsfrom2,rank_ptsfrom3,rank_ptsfromfastbreak,rank_ptsinpaint,rank_scoring,rank_steals,rank_threeptpct,rank_turnovers,rank_twoptpct,pts_visitor,pts_home,home_game,back_to_back
0,7,23,15,16,5,11,15,25,23,18,30,29,18,3,29,2,10,13,21,9,2,26,13,16,4,15,21,5,21,11,5,98,92,False,False
1,5,26,12,16,3,12,16,15,24,20,27,29,17,3,26,6,8,17,16,4,7,22,12,13,6,10,22,5,20,11,8,87,101,True,True
2,3,20,10,16,3,13,15,16,25,18,26,29,15,3,24,6,6,19,11,4,10,26,12,14,6,12,17,3,18,11,8,121,115,False,False
3,3,18,15,16,3,9,7,22,27,14,24,27,17,7,24,6,10,19,10,6,16,26,9,14,6,8,13,2,12,14,5,99,114,True,True
4,2,20,11,16,2,4,6,21,24,14,24,25,17,9,25,11,10,23,11,3,18,27,10,7,6,8,6,2,9,13,7,117,107,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12217,13,30,27,14,30,23,22,10,11,26,1,20,4,25,24,7,29,17,25,11,18,17,7,24,26,14,9,15,29,16,17,112,110,False,False
12218,14,30,27,15,30,22,22,10,12,26,1,23,4,21,25,6,29,17,25,12,17,17,6,24,26,14,9,15,29,16,18,112,128,True,False
12219,14,30,27,14,30,22,22,10,14,26,1,25,4,21,24,6,29,18,25,12,16,17,6,24,27,13,7,15,28,17,18,110,111,False,True
12220,16,30,25,15,30,21,22,10,13,25,1,25,4,23,22,8,29,19,26,9,16,18,7,24,26,13,7,14,29,17,18,125,111,True,False


In [9]:
y_home_pts = df["pts_home"]
y_away_pts = df["pts_visitor"]

In [10]:
df1 = df
del df1["pts_home"]
dropping = "pts_visitor"
X = df1.drop(dropping, axis=1)

In [11]:
(X_train, X_test, 
 y_home_pts_train, y_home_pts_test, 
 y_away_pts_train, y_away_pts_test) = train_test_split(
    X, y_home_pts, y_away_pts,
    test_size=0.2, random_state=13
)

In [12]:
df.columns

Index(['rank_assists', 'rank_blocks', 'rank_deffeciency', 'rank_drebounds',
       'rank_fouls', 'rank_ft_pct', 'rank_oeffeciency', 'rank_opp_assists',
       'rank_opp_blocks', 'rank_opp_drebounds', 'rank_opp_fouls',
       'rank_opp_orebounds', 'rank_opp_ptsfastbreak', 'rank_opp_ptsfrom2',
       'rank_opp_ptsfrom3', 'rank_opp_ptsinpaint', 'rank_opp_ptspergame',
       'rank_opp_steals', 'rank_opp_threepointpct', 'rank_opp_turnovers',
       'rank_opp_twopointpct', 'rank_orebounds', 'rank_ptsfrom2',
       'rank_ptsfrom3', 'rank_ptsfromfastbreak', 'rank_ptsinpaint',
       'rank_scoring', 'rank_steals', 'rank_threeptpct', 'rank_turnovers',
       'rank_twoptpct', 'pts_visitor', 'home_game', 'back_to_back'],
      dtype='object')

In [13]:
mapper = DataFrameMapper([
    (["rank_assists"], StandardScaler()),
    (["rank_blocks"], StandardScaler()),
    (["rank_deffeciency"], StandardScaler()),
    (["rank_drebounds"], StandardScaler()),
    (["rank_fouls"], StandardScaler()),
    (["rank_ft_pct"], StandardScaler()),
    (["rank_oeffeciency"], StandardScaler()),
    (["rank_opp_assists"], StandardScaler()),
    (["rank_opp_blocks"], StandardScaler()),
    (["rank_opp_drebounds"], StandardScaler()),
    (["rank_opp_fouls"], StandardScaler()),
    (["rank_opp_orebounds"], StandardScaler()),
    (["rank_opp_ptsfastbreak"], StandardScaler()),
    (["rank_opp_ptsfrom2"], StandardScaler()),
    (["rank_opp_ptsfrom3"], StandardScaler()),
    (["rank_opp_ptsinpaint"], StandardScaler()),
    (["rank_opp_ptspergame"], StandardScaler()),
    (["rank_opp_steals"], StandardScaler()),
    (["rank_opp_threepointpct"], StandardScaler()),
    (["rank_opp_turnovers"], StandardScaler()),
    (["rank_opp_twopointpct"], StandardScaler()),
    (["rank_orebounds"], StandardScaler()),
    (["rank_ptsfrom2"], StandardScaler()),
    (["rank_ptsfrom3"], StandardScaler()),
    (["rank_ptsfromfastbreak"], StandardScaler()),
    (["rank_ptsinpaint"], StandardScaler()),
    (["rank_scoring"], StandardScaler()),
    (["rank_steals"], StandardScaler()),
    (["rank_threeptpct"], StandardScaler()),
    (["rank_turnovers"], StandardScaler()),
    (["rank_twoptpct"], StandardScaler()),
    ("home_game", LabelBinarizer()),
    ("back_to_back", LabelBinarizer())],df_out=True)

In [14]:
Z_train = mapper.fit_transform(X_train)
Z_test = mapper.transform(X_test)

In [15]:
data_input = Input(shape=(Z_train.shape[1],), dtype='int64')

x = Dense(50, activation='relu')(data_input)
x = Dense(200, activation='relu')(x)
x = Dense(50, activation='relu')(x)

home_pts_prediction = Dense(1, name='pts_home')(x)
away_pts_prediction = Dense(1, name='pts_away')(x)
model = Model(data_input, [home_pts_prediction, away_pts_prediction])

In [16]:
model.compile(
    optimizer='adam',
    loss={'pts_home': 'mse', 'pts_away': 'mse'},
)

early_stop = EarlyStopping(monitor='loss', patience=10, verbose=0, mode='min')

model.fit(Z_train, [y_home_pts_train, y_away_pts_train], 
    epochs=10, batch_size=8, callbacks=[early_stop])

Epoch 1/10
1223/1223 [==============================] - 2s 845us/step - loss: 4342.3399 - pts_home_loss: 2358.9113 - pts_away_loss: 1983.4304
Epoch 2/10
1223/1223 [==============================] - 1s 859us/step - loss: 382.7001 - pts_home_loss: 191.8330 - pts_away_loss: 190.8671
Epoch 3/10
1223/1223 [==============================] - 1s 851us/step - loss: 347.7038 - pts_home_loss: 173.0195 - pts_away_loss: 174.6844
Epoch 4/10
1223/1223 [==============================] - 1s 850us/step - loss: 325.1614 - pts_home_loss: 161.9963 - pts_away_loss: 163.1650
Epoch 5/10
1223/1223 [==============================] - 1s 892us/step - loss: 330.4727 - pts_home_loss: 164.4490 - pts_away_loss: 166.0237
Epoch 6/10
1223/1223 [==============================] - 1s 868us/step - loss: 315.1151 - pts_home_loss: 157.5847 - pts_away_loss: 157.5304
Epoch 7/10
1223/1223 [==============================] - 1s 843us/step - loss: 311.1094 - pts_home_loss: 157.8550 - pts_away_loss: 153.2544
Epoch 8/10
1223/1223 [==

In [17]:
X_test.sample(1).to_dict(orient='list')

{'back_to_back': [False],
 'home_game': [True],
 'rank_assists': [15],
 'rank_blocks': [18],
 'rank_deffeciency': [14],
 'rank_drebounds': [10],
 'rank_fouls': [16],
 'rank_ft_pct': [24],
 'rank_oeffeciency': [7],
 'rank_opp_assists': [13],
 'rank_opp_blocks': [2],
 'rank_opp_drebounds': [17],
 'rank_opp_fouls': [1],
 'rank_opp_orebounds': [17],
 'rank_opp_ptsfastbreak': [24],
 'rank_opp_ptsfrom2': [14],
 'rank_opp_ptsfrom3': [9],
 'rank_opp_ptsinpaint': [12],
 'rank_opp_ptspergame': [11],
 'rank_opp_steals': [7],
 'rank_opp_threepointpct': [8],
 'rank_opp_turnovers': [19],
 'rank_opp_twopointpct': [8],
 'rank_orebounds': [20],
 'rank_ptsfrom2': [20],
 'rank_ptsfrom3': [8],
 'rank_ptsfromfastbreak': [10],
 'rank_ptsinpaint': [27],
 'rank_scoring': [6],
 'rank_steals': [14],
 'rank_threeptpct': [4],
 'rank_turnovers': [7],
 'rank_twoptpct': [7]}

In [18]:
X_new = pd.DataFrame({
    'back_to_back': [False],
    'home_game': [False],
    'rank_assists': [6],
    'rank_blocks': [1],
    'rank_deffeciency': [3],
    'rank_drebounds': [13],
    'rank_fouls': [14],
    'rank_ft_pct': [25],
    'rank_oeffeciency': [4],
    'rank_opp_assists': [5],
    'rank_opp_blocks': [3],
    'rank_opp_drebounds': [1],
    'rank_opp_fouls': [10],
    'rank_opp_orebounds': [12],
    'rank_opp_ptsfastbreak': [29],
    'rank_opp_ptsfrom2': [9],
    'rank_opp_ptsfrom3': [3],
    'rank_opp_ptsinpaint': [7],
    'rank_opp_ptspergame': [7],
    'rank_opp_steals': [21],
    'rank_opp_threepointpct': [5],
    'rank_opp_turnovers': [7],
    'rank_opp_twopointpct': [8],
    'rank_orebounds': [13],
    'rank_ptsfrom2': [2],
    'rank_ptsfrom3': [21],
    'rank_ptsfromfastbreak': [2],
    'rank_ptsinpaint': [2],
    'rank_scoring': [7],
    'rank_steals': [6],
    'rank_threeptpct': [11],
    'rank_turnovers': [15],
    'rank_twoptpct': [3]
})

Z_new = mapper.transform(X_new)
model.predict(Z_new)

[array([[109.35747]], dtype=float32), array([[118.77655]], dtype=float32)]

In [19]:
from sklearn.metrics import r2_score

y_hat_home_score = model.predict(Z_test)[0][:, 0]
y_hat_away_score = model.predict(Z_test)[1][:, 0]
preds = pd.DataFrame({
    'pts_home': y_home_pts_test,
    'pts_home_predicted': y_hat_home_score,
    'pts_away': y_away_pts_test,
    'pts_away_predicted': y_hat_away_score,
})

print(r2_score(preds['pts_home'], preds['pts_home_predicted']))
print(r2_score(preds['pts_away'], preds['pts_away_predicted']))

0.0028261154336653993
-0.11128622833994295
